In [1]:
import pandas as pd
#import utm
import numpy as np
import rasterio
#from meteostat import Stations, Monthly, Point
from datetime import datetime
import random
#from geopy.distance import geodesic as GD
import json
from scipy.stats import norm
import requests

In [25]:
class CropModel:
    
    def __init__(self):
        
        # General variables and configuration
        self.CropLabelName = ['ri1','ri2','ri3','ri4', 'mp', 'cf', 'sc', 'op', 'rb']
        self.FieldLatitude = 'lats'
        self.FieldLongitude = 'lons'
        
        # Base yield variables
        self.BaseYieldFilePath = 'yield_maps/'
        self.BaseYieldVarianceFactor = 0.025 # Percent to variance
        self.BaseYieldAdditionalFactor = 0.1 # percent to additional 
        
        # Disaster variables
        self.DisasterRiskMapFile = 'yield_maps/risk_map.tif'
        self.DisasterType = ['drought', 'flood']
        self.DisasterFloodReductionFactor = [0.45, 0.65] # Leftover ratio
        self.DisasterDroughtReductionFactor = [0.68, 0.83] # Leftover ratio
        
        # Rain variables
        self.OptimalRainForRice = [1500,600]  # optimal value, stdev
        self.OptimalRainForMaize = [700,200]  # optimal value, stdev
        self.OptimalRainForCassava = [1100,200]  # optimal value, stdev
        self.OptimalRainForSugarcane = [1400,200]  # optimal value, stdev
        self.OptimalRainForOilpalm = [1800,400]  # optimal value, stdev
        self.OptimalRainForRubber = [1800,300]  # optimal value, stdev
        
        
        # Temperatire variables
        self.OptimalTemperatureForRice = [25, 6] # optimal value, stdev
        self.OptimalTemperatureForMaize = [28.5, 10] # optimal value, stdev
        self.OptimalTemperatureForCassava = [25.5, 10] # optimal value, stdev
        self.OptimalTemperatureForSugarcane = [29, 6] # optimal value, stdev
        self.OptimalTemperatureForOilpalm = [27, 8] # optimal value, stdev
        self.OptimalTemperatureForRubber = [27.5, 9] # optimal value, stdev
        
        
        self.OptimalRainReductionFactors = [self.OptimalRainForRice]*4 + [self.OptimalRainForMaize] + [self.OptimalRainForCassava] + \
                                           [self.OptimalRainForSugarcane] + [self.OptimalRainForOilpalm] + [self.OptimalRainForRubber]
        
        self.OptimalTemperatureFactors = [self.OptimalTemperatureForRice]*4 + [self.OptimalTemperatureForMaize] + [self.OptimalTemperatureForCassava] + \
                                         [self.OptimalTemperatureForSugarcane] + [self.OptimalTemperatureForOilpalm] + [self.OptimalTemperatureForRubber]

        
    def fetch_agent_data(self, data_path):
        
        r=requests.get(f"http://13.229.114.44:1198/api/v1/farmer/agents?offset={offset}&limit={limit}", \
                       headers={'accept':'application/json', \
                                'X-Session-Token': token})
        
        data = r.json()['data']
        
        df = pd.DataFrame.from_dict(data)
        
        return df
        
        
    
    def convert_response_data_to_dataframe_0(self, response):
    
        data = response['data']

        df = pd.DataFrame.from_dict(data)

        return df
    
    def get_baseyield_1(self, agent_data):
        
        # make latlon set
        coordinateList = [(agent_data[self.FieldLongitude][i], agent_data[self.FieldLatitude][i]) for i in range(len(agent_data))]
        
        # for each crop
        for cropName in self.CropLabelName:
            
            with rasterio.open(self.BaseYieldFilePath+'yield_' + cropName +'.tif') as src:
                
                # sampling yield value from baseyield raster
                agent_data['baseyld_'+cropName] = [x[0] for x in src.sample(coordinateList)]
        
        # which one is not in yield map -> give NoValue
        agent_data = agent_data.replace(-9999, np.nan)
        
        return agent_data
    
    def vary_baseyield_2(self, agent_data):
        
        # Make N length of leftover yield
        randomFactorList = [random.uniform(1.0-self.BaseYieldVarianceFactor, 1.0+self.BaseYieldVarianceFactor) for i in range(len(agent_data))]
        
        for cropName in self.CropLabelName:
            
            # Calculate leftover yield for every crop
            agent_data['varyld_'+cropName] = agent_data['baseyld_'+cropName] * randomFactorList * (1.0+self.BaseYieldAdditionalFactor)
        
        return agent_data
    
    def shock_temp_3(self, agent_data, temp_range):
        
        ReductionFactorForTemp = 5 #5
        
        # sampling a temperature value in minmax range
        temperatureFromRandom = random.uniform(temp_range[0], temp_range[1])
        
        # Generate Shock factor from temp
        shockFactorsListForEachCrop = []
        
        for cropName, temperatureFactorAtCrop in zip(self.CropLabelName, self.OptimalTemperatureFactors):
            
            shockFactorsListForEachCrop.append(self.get_shock_factor(temperatureFromRandom, temperatureFactorAtCrop[0], temperatureFactorAtCrop[1], ReductionFactorForTemp))
        
        # for each crop
        for i, cropName in enumerate(self.CropLabelName):
            
            agent_data['tmpshk_'+cropName] = agent_data['varyld_'+cropName] * shockFactorsListForEachCrop[i]
        
        return agent_data
    
    def shock_rain_4(self, agent_data, rain_range):
        
        ReductionFactorForRain = 20 #20
        
        # sampling a temperature value in minmax range
        rainFromRandom = random.uniform(rain_range[0], rain_range[1])
        
        # Generate Shock factor from temp
        shockFactorsListForEachCrop = []
        
        for cropName, rainFactorAtCrop in zip(self.CropLabelName, self.OptimalRainReductionFactors):
            
            shockFactorsListForEachCrop.append(self.get_shock_factor(rainFromRandom, rainFactorAtCrop[0], rainFactorAtCrop[1], ReductionFactorForRain))
        
        # for each crop
        for i, cropName in enumerate(self.CropLabelName):
            
            agent_data['ranshk_'+cropName] = agent_data['tmpshk_'+cropName] * shockFactorsListForEachCrop[i]
        
        return agent_data
    
    def shock_disaster_5(self, agent_data):
        
        coordinateList = [(agent_data[self.FieldLongitude][i], agent_data[self.FieldLatitude][i]) for i in range(len(agent_data))]
        
        with rasterio.open(self.DisasterRiskMapFile) as src:
                
            # sampling yield value from baseyield raster
            sampleRiskValues = [x[0] for x in src.sample(coordinateList)]
        
        is_disaster = [1 if random.random() < sampleRiskValues[i] else 0 for i in range(len(sampleRiskValues))]

        disaster_type = random.choice(self.DisasterType)
        
        if disaster_type == 'flood':
            max_severe_list = [random.uniform(self.DisasterFloodReductionFactor[0], self.DisasterFloodReductionFactor[1]) for i in range(len(is_disaster))]
        else:
            max_severe_list = [random.uniform(self.DisasterDroughtReductionFactor[0], self.DisasterDroughtReductionFactor[1]) for i in range(len(is_disaster))]
        
        severity_percent = [random.random() if is_disaster[i] else 0 for i in range(len(sampleRiskValues))]
        
        left_over_yield_ratio = [1-((1-max_severe_list[i])* severity_percent[i]) if is_disaster[i] else 1. for i in range(len(sampleRiskValues))]

        # for each crop
        for i, cropName in enumerate(self.CropLabelName):
            
            agent_data['dis_'+cropName] = agent_data['ranshk_'+cropName] * left_over_yield_ratio
            
        agent_data['dis_severity'] = severity_percent
        
        return agent_data
    
    def clean_agent_data(self, agent_data, is_disaster):
        
        if not is_disaster:
            
            output_columns = ['ids'] + ['ranshk_'+cropName for cropName in self.CropLabelName]
            clean_columns = ['ids'] + [cropName for cropName in self.CropLabelName]
        else:
            output_columns = ['ids'] + ['dis_'+cropName for cropName in self.CropLabelName] +\
                             ['dis_severity']
            clean_columns = ['ids'] + [cropName for cropName in self.CropLabelName] +\
                             ['severity']
        
        output_dataframe = agent_data[output_columns]
        
        output_dataframe.columns = clean_columns
        
        
        return output_dataframe
            
        
    def convert_result_to_response_6(self, data):
        
        result = {}
        
        for col in data.columns:
            
            result[col] = data[col].values.tolist()
            
        return result
    
    # A function to calculate leftover yield variance from optimal enviroment factors of each crop
    # x = current env (tmp or rain), mu = optimal env, sigma = stdev env, reduction_factor= multiplication number for preventing over reduce yield
    def get_shock_factor(self, x, mu, sigma, reduction_factor):
        
        return (1-norm.cdf(mu+abs(mu-x), mu, sigma*reduction_factor))*2

In [26]:
model = CropModel()

In [69]:
data = json.load(open('response_1675134414744.json'))

In [70]:
data = pd.DataFrame.from_dict(data['data'])

In [71]:
data

,ids,lats,lons,districts
0,1,15.582785,100.040045,2
1,2,15.582785,100.040045,2
2,3,15.582785,100.040045,2
3,4,15.575076,100.100648,2
4,5,15.853955,99.647099,4
...,...,...,...,...
995,996,15.381504,100.210354,9
996,997,15.378502,100.211564,9
997,998,15.381504,100.210354,9
998,999,15.378672,100.210980,9


In [72]:
data = model.get_baseyield_1(data)

In [73]:
data = model.vary_baseyield_2(data)

In [74]:
data

,ids,lats,lons,districts,baseyld_ri1,baseyld_ri2,baseyld_ri3,baseyld_ri4,baseyld_mp,baseyld_cf,...,baseyld_rb,varyld_ri1,varyld_ri2,varyld_ri3,varyld_ri4,varyld_mp,varyld_cf,varyld_sc,varyld_op,varyld_rb
0,1,15.582785,100.040045,2,363.0,712.0,650.506042,600.0,1192.026855,3714.885010,...,156.576553,407.974779,800.214993,731.102090,674.338477,1339.715957,4175.149832,741.828643,1340.483560,175.975991
1,2,15.582785,100.040045,2,363.0,712.0,650.506042,600.0,1192.026855,3714.885010,...,156.576553,404.751388,793.892530,725.325685,669.010559,1329.130922,4142.162162,735.967488,1329.892460,174.585612
2,3,15.582785,100.040045,2,363.0,712.0,650.506042,600.0,1192.026855,3714.885010,...,156.576553,407.168661,798.633848,729.657506,673.006052,1337.068813,4166.900156,740.362864,1337.834900,175.628280
3,4,15.575076,100.100648,2,363.0,712.0,648.250977,600.0,1233.748291,3692.024658,...,158.222916,397.793906,780.245898,710.386468,657.510588,1352.004273,4045.908839,726.231694,1191.166070,173.388737
4,5,15.853955,99.647099,4,363.0,712.0,664.299561,600.0,1253.970459,3741.654053,...,158.953400,400.352073,785.263571,732.654838,661.738964,1383.001854,4126.663795,740.189201,1498.597899,175.309430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,15.381504,100.210354,9,363.0,712.0,663.501587,600.0,1257.830566,3635.557617,...,147.132156,390.472366,765.885192,713.716348,645.408869,1353.025006,3910.701885,703.434478,1560.674858,158.267331
996,997,15.378502,100.211564,9,363.0,712.0,663.501587,600.0,1257.830566,3635.557617,...,147.132156,394.469972,773.726226,721.023285,652.016483,1366.877103,3950.739151,710.636150,1576.652847,159.887652
997,998,15.381504,100.210354,9,363.0,712.0,663.501587,600.0,1257.830566,3635.557617,...,147.132156,401.066684,786.665231,733.080939,662.920138,1389.735355,4016.807263,722.520100,1603.019173,162.561449
998,999,15.378672,100.210980,9,363.0,712.0,663.501587,600.0,1257.830566,3635.557617,...,147.132156,409.027637,802.280103,747.632193,676.078738,1417.320837,4096.538678,736.861727,1634.838222,165.788204


In [63]:
data = model.shock_temp_3(data, (15, 35))

In [64]:
data = model.shock_rain_4(data, (1800, 2200))

In [65]:
data = model.shock_disaster_5(data)

In [66]:
data

,ids,lats,lons,districts,baseyld_ri1,baseyld_ri2,baseyld_ri3,baseyld_ri4,baseyld_mp,baseyld_cf,...,dis_ri1,dis_ri2,dis_ri3,dis_ri4,dis_mp,dis_cf,dis_sc,dis_op,dis_rb,dis_severity
0,1,15.582785,100.040045,2,363.0,712.0,650.506042,600.0,1192.026855,3714.885010,...,249.424784,489.229879,446.976113,412.272370,655.476042,2385.458551,357.615518,848.238290,112.457358,0.788577
1,2,15.582785,100.040045,2,363.0,712.0,650.506042,600.0,1192.026855,3714.885010,...,295.401679,579.410456,529.367981,488.267238,776.301056,2825.174186,423.535395,1004.595497,133.186814,0.080588
2,3,15.582785,100.040045,2,363.0,712.0,650.506042,600.0,1192.026855,3714.885010,...,294.709565,578.052921,528.127694,487.123248,774.482216,2818.554922,422.543070,1002.241773,132.874763,0.000000
3,4,15.575076,100.100648,2,363.0,712.0,648.250977,600.0,1233.748291,3692.024658,...,265.038236,519.854612,473.309354,438.079729,720.885474,2519.184759,381.532971,821.434261,120.753425,0.447105
4,5,15.853955,99.647099,4,363.0,712.0,664.299561,600.0,1253.970459,3741.654053,...,253.327441,496.884677,463.595889,418.723043,700.326756,2440.241235,369.308554,981.466237,115.950760,0.706646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,15.381504,100.210354,9,363.0,712.0,663.501587,600.0,1257.830566,3635.557617,...,296.085402,580.751532,541.193207,489.397359,821.051347,2771.245000,420.587936,1224.867798,125.442915,0.000000
996,997,15.378502,100.211564,9,363.0,712.0,663.501587,600.0,1257.830566,3635.557617,...,292.519965,573.758167,534.676200,483.504074,811.164311,2737.873882,415.523248,1210.118035,123.932341,0.000000
997,998,15.381504,100.210354,9,363.0,712.0,663.501587,600.0,1257.830566,3635.557617,...,302.332666,593.005119,552.612132,499.723415,838.375148,2829.716979,429.462145,1250.711938,128.089702,0.000000
998,999,15.378672,100.210980,9,363.0,712.0,663.501587,600.0,1257.830566,3635.557617,...,215.429853,422.551117,393.768731,356.082402,597.391732,2016.340215,306.017104,891.206010,91.271466,0.883595


In [ ]:
data

In [67]:
data = model.clean_agent_data(data, is_disaster=True)

In [68]:
data

,ids,ri1,ri2,ri3,ri4,mp,cf,sc,op,rb,severity
0,1,249.424784,489.229879,446.976113,412.272370,655.476042,2385.458551,357.615518,848.238290,112.457358,0.788577
1,2,295.401679,579.410456,529.367981,488.267238,776.301056,2825.174186,423.535395,1004.595497,133.186814,0.080588
2,3,294.709565,578.052921,528.127694,487.123248,774.482216,2818.554922,422.543070,1002.241773,132.874763,0.000000
3,4,265.038236,519.854612,473.309354,438.079729,720.885474,2519.184759,381.532971,821.434261,120.753425,0.447105
4,5,253.327441,496.884677,463.595889,418.723043,700.326756,2440.241235,369.308554,981.466237,115.950760,0.706646
...,...,...,...,...,...,...,...,...,...,...,...
995,996,296.085402,580.751532,541.193207,489.397359,821.051347,2771.245000,420.587936,1224.867798,125.442915,0.000000
996,997,292.519965,573.758167,534.676200,483.504074,811.164311,2737.873882,415.523248,1210.118035,123.932341,0.000000
997,998,302.332666,593.005119,552.612132,499.723415,838.375148,2829.716979,429.462145,1250.711938,128.089702,0.000000
998,999,215.429853,422.551117,393.768731,356.082402,597.391732,2016.340215,306.017104,891.206010,91.271466,0.883595
